In [17]:
import duckdb
import pandas as pd
import torch
from transformers import *
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk


In [18]:
#filtering the ontology for level_3  research method and data analysis method.
ontology_df = pd.read_csv('/Users/sheillapurwandiary/spurwand/BIPM/Semester2/2TWSM/OntologyProject/ontology_streamlit_app/app/data/ontology.csv')

#filtering the ontology for level_3  research method and data analysis method.

synonyms_df = pd.read_csv('/Users/sheillapurwandiary/spurwand/BIPM/Semester2/2TWSM/OntologyProject/ontology_streamlit_app/app/data/synonyms.csv')

synonyms_df.info()
ontology_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31770 entries, 0 to 31769
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   synonym  31770 non-null  object
dtypes: object(1)
memory usage: 248.3+ KB


,ent_id,definition,label,parent,level,related,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
0,research method,A research method is a systematic plan for con...,NaN,methodological entity,3.0,NaN,entity,methodological entity,research method,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,conceptual method,A conceptual method refers to a systematic app...,CONCEPTUAL_METHOD,research method,4.0,NaN,entity,methodological entity,research method,conceptual method,NaN,NaN,NaN,NaN,NaN,NaN
2,literature study,A literature study is a systematic and compreh...,CONCEPTUAL_METHOD,conceptual method,5.0,NaN,entity,methodological entity,research method,conceptual method,literature study,NaN,NaN,NaN,NaN,NaN
3,structured literature research,Structured literature research is a methodical...,CONCEPTUAL_METHOD,literature study,6.0,NaN,entity,methodological entity,research method,conceptual method,literature study,structured literature research,NaN,NaN,NaN,NaN
4,literature sample,A literature sample refers to a specific piece...,CONCEPTUAL_METHOD,structured literature research,7.0,NaN,entity,methodological entity,research method,conceptual method,literature study,structured literature research,literature sample,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,questionable research practice,Questionable research practice refers to the a...,ANALYSIS_METHOD,quantitative analysis,5.0,NaN,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,NaN,NaN,NaN,NaN,NaN
895,data dredging,Data dredging refers to the practice of conduc...,ANALYSIS_METHOD,questionable research practice,6.0,NaN,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,data dredging,NaN,NaN,NaN,NaN
896,p hacking,P hacking is the manipulative practice of rean...,ANALYSIS_METHOD,data dredging,7.0,NaN,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,data dredging,p hacking,NaN,NaN,NaN
897,HARKing,HARKing refers to the practice of formulating ...,ANALYSIS_METHOD,questionable research practice,6.0,NaN,entity,methodological entity,data analysis method,quantitative analysis,questionable research practice,HARKing,NaN,NaN,NaN,NaN


In [19]:
synonyms_df

,synonym
0,QDA Miner Lite system
1,audio coding
2,manually coded
3,code all statement
4,coded each interview
...,...
31765,galvanometer
31766,galvanic skin activity
31767,psychogalvanic activity
31768,electrodermal response


In [20]:
print('Before ', synonyms_df.shape)
synonyms_df.drop_duplicates(inplace=True)
print('After: ', synonyms_df.shape)

#resetting index to avoid issues
synonyms_df.reset_index(drop=True, inplace=True)

Before  (31770, 1)
After:  (31770, 1)


In [21]:
# Creating a copy of the ontology_df and selecting the 'parent' column
data_ontology = ontology_df.copy()
data_ontology = data_ontology[['parent']]
data_ontology['text'] = data_ontology['parent']
del(data_ontology['parent'])

# Creating a copy of the synonyms_df and selecting the 'synonym' column
data_synonyms = synonyms_df.copy()
data_synonyms['text'] = data_synonyms['synonym']
del(data_synonyms['synonym'])


In [22]:
print('Before ', data_ontology.shape)
data_ontology.drop_duplicates(inplace=True)
print('After: ', data_ontology.shape)

#resetting index to avoid issues
data_ontology.reset_index(drop=True, inplace=True)

Before  (899, 1)
After:  (220, 1)


In [23]:
data_synonyms


,text
0,QDA Miner Lite system
1,audio coding
2,manually coded
3,code all statement
4,coded each interview
...,...
31765,galvanometer
31766,galvanic skin activity
31767,psychogalvanic activity
31768,electrodermal response


In [24]:
data_ontology

,text
0,methodological entity
1,research method
2,conceptual method
3,literature study
4,structured literature research
...,...
215,statistical test assumption
216,homoskedasticity test
217,odds ratio homogeneity test
218,questionable research practice


In [25]:
# Unique entries in data_synonyms
unique_synonyms = data_synonyms['text'].unique()
print(f"Unique entries in data_synonyms: {len(unique_synonyms)}")

# Unique entries in data_ontology
unique_ontology = data_ontology['text'].unique()
print(f"Unique entries in data_ontology: {len(unique_ontology)}")

# Find common entries between the two dataframes
common_entries = set(unique_synonyms).intersection(set(unique_ontology))
print(f"Common entries between data_synonyms and data_ontology: {len(common_entries)}")
print(f"Common entries: {common_entries}")


Unique entries in data_synonyms: 31770
Unique entries in data_ontology: 220
Common entries between data_synonyms and data_ontology: 4
Common entries: {'archival database', 'quantitative analysis', 'research method', 'qualitative analysis'}


In [26]:
data_synonyms_list = data_synonyms["text"].to_list()
data_ontology_list = data_ontology["text"].to_list()

In [27]:
data_synonyms_list

['QDA Miner Lite system',
 'audio coding',
 'manually coded',
 'code all statement',
 'coded each interview',
 'video coding',
 'code the interview',
 'label all statement',
 'narrative inquiry',
 'quantitative research',
 'IBM SPSS statistics',
 'R Open',
 'StyleGAN2',
 'Transformerbased model',
 'Gpt3',
 'DALL E',
 'supervised algorithm',
 'ID3 algorithm',
 'iterative dichotomiser 3',
 'ID3',
 'fully self referential learner',
 'clustering investigation',
 'clustering approach',
 'association rule mining',
 'self supervised analysis',
 'non contrastive self supervised learning',
 'LDA method',
 'POS tag',
 'emotion artificial intelligence',
 'opinion mining',
 'aspect based opinion mining',
 'extreme values model',
 'extreme value technique',
 'extreme value algorithm',
 'feature transformation',
 'imputation',
 'last observation data imputation',
 'regression imputation',
 'latent class model',
 'multi level structural equation model',
 'SEMModComp',
 'LISREL',
 'lavaan',
 'structur

In [28]:
data_ontology_list

['methodological entity',
 'research method',
 'conceptual method',
 'literature study',
 'structured literature research',
 'literature sample',
 'bibliometric software',
 'literature database',
 'Web of Science Core Collection',
 'academic journal list',
 'individual IS journal',
 'bibliometric analysis',
 'simulation',
 'stochastic simulation',
 'Monte Carlo simulation',
 'Markov chain Monte Carlo',
 'computational algorithm',
 'mathematical model',
 'economic model',
 'stochastic model',
 'Markov model',
 'optimization model',
 'discrete optimization',
 'integer programming',
 'integer linear programming',
 'combinatorial optimization',
 'traveling purchaser problem',
 'multiple criteria decision making',
 'multi objective optimization',
 'design science',
 'design methodology',
 'design process',
 'conceptual modelling',
 'maturity model',
 'capability maturity model',
 'data modeling',
 'database modeling',
 'method engineering',
 'business process modeling',
 'business process m

In [31]:

# Combine both lists to create a corpus
combined_data = data_synonyms_list + data_ontology_list

# Tokenize the text (simple whitespace tokenization)
tokenized_data = [text.split() for text in combined_data]

# Train the Word2Vec model
model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

# Get vectors for each word in data_synonyms_list and data_ontology_list
embeddings_synonyms = {word: model.wv[word] for text in data_synonyms_list for word in text.split()}
embeddings_ontology = {word: model.wv[word] for text in data_ontology_list for word in text.split()}

# Define the get_embeddings function
def get_embeddings(term):
    tokens = term.split()
    embeddings = {}
    for token in tokens:
        if token in model.wv:
            embeddings[token] = model.wv[token]
    return embeddings

# Define the is_synonym function
def is_synonym(new_term):
    """
    Compare a new_term with the list of synonyms and ontology terms stored using cosine similarity.

    Args:
        new_term: a single term to be compared
    
    Returns:
        A string that tells if the term is a synonym to x word and the score or if there was not a synonym in the list
    """
    new_term_embds = get_embeddings(new_term)
    if not new_term_embds:
        return f"No embeddings found for the new term: {new_term}"

    similarity_scores = {}
    overall_embeddings = {**embeddings_synonyms, **embeddings_ontology}
    
    for term, embedding in overall_embeddings.items():
        for new_term_token, new_term_embedding in new_term_embds.items():
            similarity = cosine_similarity(new_term_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
            similarity_scores[term] = similarity
    
    sorted_similarity_scores = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True))
    
    if sorted_similarity_scores and list(sorted_similarity_scores.values())[0] > 0.5:
        top_synonym = list(sorted_similarity_scores.keys())[0]
        return f"'{new_term}' is synonym to '{top_synonym}' with a similarity score of {list(sorted_similarity_scores.values())[0]:.2f}"
    else:
        return f"No close synonyms found in the list for the term: {new_term}"

# Example usage
print(is_synonym("apple"))
print(is_synonym("integer programming"))
print(is_synonym("questionare"))
print(is_synonym("questionable research practice"))
print(is_synonym("education"))

No embeddings found for the new term: apple
'integer programming' is synonym to 'programming' with a similarity score of 1.00
No embeddings found for the new term: questionare
'questionable research practice' is synonym to 'practice' with a similarity score of 1.00
No embeddings found for the new term: education


: 